<a href="https://colab.research.google.com/github/exp3012/IT_SEC_J_2509/blob/main/denselayer_minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from PIL import Image # [수정됨] .convert('RGB')를 사용하기 위해 import 확인

# --- 1. 환경 설정 및 데이터 준비 ---
# MNIST 데이터를 디렉토리 구조로 저장하기 위한 경로 설정
DATA_DIR = './mnist_data_dir'
os.makedirs(DATA_DIR, exist_ok=True)
print(f"데이터 저장 경로: {DATA_DIR}")

# 기존 디렉토리 구조가 있다면 삭제하고 새로 생성
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(DATA_DIR)

# Keras에서 MNIST 데이터셋 로드 (NumPy 배열 형태)
print("MNIST 데이터셋 로드 중...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(f"학습 이미지 크기: {x_train.shape}")
print(f"테스트 이미지 크기: {x_test.shape}")

# 디렉토리에 이미지 파일로 저장하는 함수 (0-9까지의 클래스별 폴더 생성)
def save_images_to_directory(images, labels, base_dir, split_name):
    # 'train' 또는 'test' 폴더 생성
    split_dir = os.path.join(base_dir, split_name)
    os.makedirs(split_dir, exist_ok=True)

    # 클래스(0~9)별 폴더 생성
    for i in range(10):
        os.makedirs(os.path.join(split_dir, str(i)), exist_ok=True)

    # 이미지 파일 저장
    print(f"{split_name} 이미지 파일 저장 시작...")
    for i in range(len(images)):
        label = labels[i]

        image_name = f"{i:05d}.png"
        file_path = os.path.join(split_dir, str(label), image_name)

        # [수정됨] 1/3: 흑백(L) 이미지를 3채널(RGB)로 변환하여 저장
        # 흑백 이미지를 8비트 그레이스케일로 변환
        img = Image.fromarray(images[i].astype(np.uint8))
        # 1채널 이미지를 R,G,B 채널에 복제하여 3채널 이미지로 변환
        img = img.convert('RGB')
        img.save(file_path)

    print(f"{split_name} 데이터 디렉토리 구조 생성 완료.")


save_images_to_directory(x_train, y_train, DATA_DIR, 'train')
save_images_to_directory(x_test, y_test, DATA_DIR, 'test')


# --- 2. 'image_dataset_from_directory'를 이용한 데이터 로드 및 전처리 ---

BATCH_SIZE = 32
IMAGE_SIZE = (28, 28)

print("\n디렉토리에서 데이터셋 로드 중...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=os.path.join(DATA_DIR, 'train'),
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    interpolation='bilinear',
    color_mode='rgb', # [수정됨] 2/3: 'grayscale'이 아닌 'rgb' (3채널)로 로드
    batch_size=BATCH_SIZE,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=os.path.join(DATA_DIR, 'test'),
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    interpolation='bilinear',
    color_mode='rgb', # [수정됨] 2/3: 'grayscale'이 아닌 'rgb' (3채널)로 로드
    batch_size=BATCH_SIZE,
    seed=42
)

# 데이터 정규화 함수 (픽셀 값을 0과 1 사이로)
# (이미지가 (28, 28, 3)으로 로드되지만 정규화 방식은 동일함)
def normalize(image, label):
    # 픽셀 값을 0-255에서 0-1로 변환합니다.
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# 데이터셋에 정규화 적용 및 캐싱/프리페치 설정 (학습 속도 향상)
train_ds = train_ds.map(normalize).cache().prefetch(buffer_size=tf.data.AUTOTUNE) # Corrected typo
val_ds = val_ds.map(normalize).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
print("데이터셋 로드 및 전처리 완료. (3채널 RGB 모드)")


# --- 3. Dense Layer만으로 구성된 모델 정의 ---

model = Sequential([
    # [수정됨] 3/3: 입력 shape을 (28, 28, 3)으로 변경
    # (28, 28) 크기의 3채널(RGB) 이미지를
    # (28 * 28 * 3 = 2352) 크기의 1차원 벡터로 '펼칩니다'.
    Flatten(input_shape=(28, 28, 3), name='Flatten_Layer_2352_features'),

    # 은닉층 1: 512개의 뉴런을 가진 Dense Layer
    Dense(512, activation='relu', name='Dense_Hidden_512'),

    # 은닉층 2: 256개의 뉴런을 가진 Dense Layer (추가적인 복잡도 학습)
    Dense(256, activation='relu', name='Dense_Hidden_256'),

    # 출력층: 10개의 뉴런 (0~9의 숫자), Softmax 활성화 함수 (확률 분포 출력)
    Dense(10, activation='softmax', name='Output_Layer_10_Classes')
])

# --- 4. 모델 컴파일 및 학습 ---

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n모델 구조 (3채널 입력):")
model.summary() # Flatten 레이어의 Param이 0, Dense_Hidden_512의 Param이 (2352+1)*512 임을 확인

print("\n모델 학습 시작...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10 # 에포크 수: 10회 학습
)

# --- 5. 최종 평가 ---
print("\n테스트 데이터셋으로 최종 모델 평가:")
loss, accuracy = model.evaluate(val_ds)

print(f"\n최종 테스트 정확도 (Dense Layer Only, 3-Channel): {accuracy*100:.2f}%")
print("3채널(RGB) 이미지로 모델 학습이 완료되었습니다.")

데이터 저장 경로: ./mnist_data_dir
MNIST 데이터셋 로드 중...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
학습 이미지 크기: (60000, 28, 28)
테스트 이미지 크기: (10000, 28, 28)
train 이미지 파일 저장 시작...
train 데이터 디렉토리 구조 생성 완료.
test 이미지 파일 저장 시작...
test 데이터 디렉토리 구조 생성 완료.

디렉토리에서 데이터셋 로드 중...
Found 60000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
데이터셋 로드 및 전처리 완료. (3채널 RGB 모드)

모델 구조 (3채널 입력):


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Flatten_Layer_2352_features     │ (None, 2352)           │             0 │
│ (Flatten)                       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_Hidden_512 (Dense)        │ (None, 512)            │     1,204,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_Hidden_256 (Dense)        │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer_10_Classes (Dense) │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,338,634 (5.11 MB)

 Trainable params: 1,338,634 (5.11 MB)

 Non-trainable params: 0 (0.00 B)


모델 학습 시작...
Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 56s 29ms/step - accuracy: 0.9065 - loss: 0.2980 - val_accuracy: 0.9658 - val_loss: 0.1069
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9711 - loss: 0.0936 - val_accuracy: 0.9637 - val_loss: 0.1217
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9794 - loss: 0.0655 - val_accuracy: 0.9715 - val_loss: 0.1041
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9855 - loss: 0.0482 - val_accuracy: 0.9764 - val_loss: 0.1053
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9868 - loss: 0.0413 - val_accuracy: 0.9743 - val_loss: 0.1141
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 45s 24ms/step - accuracy: 0.9895 - loss: 0.0348 - val_accuracy: 0.9717 - val_loss: 0.1406
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9904 - loss: 0.0309 - val_accuracy: 0.9777 - val_loss: 0.1174
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accur

In [ ]:
# --- 6. [추가] 학습 완료된 모델 저장하기 ---
# (이 부분은 이전과 동일하게 모델을 저장합니다)
MODEL_SAVE_PATH = './my_mnist_dense_model.keras'

print(f"\n학습된 모델을 '{MODEL_SAVE_PATH}' 경로에 저장합니다...")
model.save(MODEL_SAVE_PATH)
print("모델 저장 완료.")





학습된 모델을 './my_mnist_dense_model.keras' 경로에 저장합니다...
모델 저장 완료.


In [ ]:
# --- 7. [요청대로 수정] 시각화 X, 컬러(RGB) 로드 ---

# from matplotlib.pyplot as plt # [요청] 시각화 제거로 import 안 함
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os

# --- 1. 모델 로드 ---
print(f"\n'{MODEL_SAVE_PATH}' 경로에서 모델을 로드합니다...")
loaded_model = tf.keras.models.load_model(MODEL_SAVE_PATH)
print("모델 로드 완료.")


# --- 2. 예측할 이미지 파일 경로 지정 ---
# !!! 사용자가 예측하려는 파일의 경로를 여기에 직접 입력하세요 !!!
my_img = "minst_1_2.png" # (예: "my_cat.jpg")

# (테스트용: 1.png가 없다면 임시로 하나 복사합니다.)
if not os.path.exists(my_img) and os.path.exists(os.path.join(DATA_DIR, 'test', '7', '00000.png')):
    print(f"[테스트용 알림] '{my_img}'가 없어 임시 파일을 복사합니다. 예측할 파일을 직접 업로드하세요.")
    shutil.copy(os.path.join(DATA_DIR, 'test', '7', '00000.png'), my_img)


# --- 3. 파일 존재 확인 및 예측 수행 ---
if not os.path.exists(my_img):
    print(f"\n[오류] '{my_img}' 파일을 찾을 수 없습니다!")
    print(f"Colab의 왼쪽 '파일' 탭에 '{my_img}' 파일을 업로드하거나 경로를 확인해주세요.")
else:
    print(f"\n--- 지정된 파일로 예측 시작: {my_img} ---")

    # 1. 이미지 로드 (28x28 크기, 컬러(RGB)로 로드)
    img = load_img(
        my_img,
        # color_mode='grayscale', # [요청] 흑백 처리 제거 (기본값 'rgb')
        target_size=(28, 28)
    )

    # 2. 이미지를 NumPy 배열로 변환
    # (흑백 처리를 뺐으므로 shape은 (28, 28, 3)이 됩니다)
    img_array = img_to_array(img)

    # 3. 정규화 (0~1 사이로)
    img_array = img_array / 255.0

    # 4. 배치 차원 추가
    # (shape: (1, 28, 28, 3))
    img_for_prediction = tf.expand_dims(img_array, axis=0)

    # 5. 예측 수행 (채널 수가 맞지 않아 오류가 발생할 것입니다)
    try:
        prediction_scores = loaded_model.predict(img_for_prediction)

        # 6. 결과 해석
        predicted_label = np.argmax(prediction_scores[0])
        confidence = 100 * np.max(prediction_scores[0])

        print(f"  모델 예측 결과: {predicted_label}")
        print(f"  신뢰도: {confidence:.2f}%")

    except ValueError as e:
        print("\n" + "="*30)
        print("[예측 오류 발생!]")
        print("  (이것은 정상적인 오류입니다)")
        print(f"  오류 원인: {e}")
        print("\n[해설]")
        print(f"  훈련된 모델의 입력 shape: {loaded_model.input_shape} (흑백/1채널)")
        print(f"  현재 입력된 이미지 shape: {img_for_prediction.shape} (컬러/3채널)")
        print("  -> 모델을 흑백(1채널)으로 학습시켰기 때문에 컬러(3채널) 이미지를 예측할 수 없습니다.")
        print("="*30)

    # 7. 예측 결과 시각화
    # [요청] "이미지 없는 상황" -> 시각화 코드(plt.show()) 완전 제거